Assignment 5 (50 points)

Due Date: April 22, 24 at 11:59 PM

Agents are an emerging field thats use reflection, tools, planning, and multi agent collaboration

In this assignment, we will build a research agent. We will use serverless LLM endpoints. To get started, you create an account with Together AI or Anthropic. They should provide you with a few dollars worth of credits that should be enough to complete the assignment. You are free to choose any other provider such as OpenAI, Mistral, Fireworks, OctoAI, or Groq. I encourage you to play around with different models to get a feel for how they work. For this assignment, the API usage cost should be around a couple dollars. Depending on the model you choose and how many attempts you use, it may be a couple cents. For OpenAI, Anthropic, and Mistral, double check what model you are using. The flagship models are significantly more expensive than the smaller models (pricing between models varies by 50x). For the purposes of this assignment, it is sufficient to use the smallest/cheapest models.

TogetherAI, Fireworks, OctoAI, and Groq run open source models. For these, it’s better to run the mid-large tier models. Mixtral is a good place to start. It’s good to play around with different models.

Many providers are able to use OpenAI's client library, but some do not (like Anthropic). Use whatever makes sense.

You can run this on Colab with a CPU, or locally and submit the Jupyter notebook as your submission. Since we are using third party providers for the LLMs, we will not load the model locally. If you run on Colab, take special care to not leak your API key. Here’s an example of how to properly use secrets in Colab.

Research Agent
Build an LLM-based research agent that can take a research topic, find relevant information, and generate a short summary (~1 paragraph) on the given topic.

Tools to Implement (20 points, 4 points each):
1. Topic Breakdown Tool: Create a tool that takes a broad research topic and breaks it down into smaller, more focused subtopics or subqueries. You can use an LLM to generate these subtopics based on the main topic.

2. Query Expansion Tool: Develop a tool to expand the subqueries generated by the Topic Breakdown Tool. The tool should generate related keywords, synonyms, and phrases to enhance the search results.

3. Search Tool: Create a wrapper around the You API or Brave Search API. Please note that the free tier is 1000 queries/month. Consider creating a mock while developing, and switch to actually call the You API once the agent is more stable. Additionally, consider caching the search results.

4. Critique Tool: Create a tool that critiques the summary, and offers suggestions of how to improve and potentially other relevant topics to search for.

5. Summarizer Tool (optional): Create a tool that takes some input and summarizes its content using an LLM.

Workflow (30 points)
Implement an agent workflow that uses all of these tools. In the agent workflow, the agent should be provided with all the tools and it should decide which tool to use. For the individual tool implementations, if you use a call to an LLM you do not need to provide any tools.

Sample Agent Workflow:
1. The agent receives a research topic from the user.
2. It uses the Topic Breakdown Tool to generate subtopics or subqueries.
3. The Query Expansion Tool expands the subqueries with related keywords and phrases.
4. The Search Tool uses the expanded queries and subqueries to gather relevant information from various sources.
5. The agent generates the summary incorporating the search results. (optional)
6. The agent critiques the summary, and improves the results. (optional)
7. The agent presents the final summary to the user.

The sample workflow is the minimum implementation requirement. Feel free to add more tools, add loops in the workflow, etc.



In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
import os
from google.colab import userdata

client = OpenAI(
  api_key=userdata.get('TOGETHER_API_KEY'),
  base_url='https://api.together.xyz/v1',
)

In [3]:
import openai

research_topic = "Impact of Human-Dog Interaction on Well-being"
model = "mistralai/Mistral-7B-v0.1"
model = "Qwen/Qwen1.5-4B-Chat"

def generate_subtopics(research_topic):
    messages = [
    {"role": "system",
     "content": "You are a psychologist studying the impacts of pet ownership on humans."},
    {"role": "user",
     "content": f"Generate 5 subtopics for the research topic: {research_topic}. Be Concise."}
   ]
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              max_tokens=200,
                                              stop=None,
                                              temperature=0.8)
    subtopics =  response.choices[0].message.content
    return subtopics

In [4]:
def expand_queries(subtopics):
    expanded_queries = [
        {"role": "system", "content": "You are a psychologist studying the benefits of pet dog ownership."},
        {"role": "user", "content": f"Generate 5 alternate phrases (IN ENGLISH) for each of the following subtopics:{subtopics} that have the same meaning. Your response should follow the below payload: {{'query':query, 'keywords_phrases':[]}}"}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=expanded_queries,
        max_tokens=500,
        stop=None,
        temperature=0.7,
    )
    expanded_queries = response.choices[0].message.content
    return expanded_queries

In [71]:
import requests
#userdata.get("YOU-API-KEY")
def get_ai_snippets_for_query(query):
    headers = {"X-API-KEY": userdata.get("YOU-API-KEY")}
    params = {"query": query}
    return requests.get(
        f"https://api.ydc-index.io/search",
        params=params,
        headers=headers,
    ).json()

In [72]:
def summary_generator(search_result):
    messages = [
        {"role": "system", "content": "You are an Psychologist conducting research and gathering relevant research."},
        {"role": "user", "content": f"Please Summarize the following research:\n\n{search_result}."}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500,
        stop=None,
        temperature=0.7,
    )
    summ = response.choices[0].message.content
    return summ

In [73]:
def evaluate_search(summary):
    messages = [
        {"role": "system", "content": "You are an Psychologist who is peer reviewing an editorial for a monthly magazine."},
        {"role": "user", "content": f"Please critique the following summarie:\n\n{summary}. Identify shortcomings in the summary, provide constructive feedback and identify topics that would improve the quality of the summary"}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500,
        stop=None,
        temperature=0.7,
    )
    critique = response.choices[0].message.content
    return critique

In [74]:
def summary_generator_improve(summary, feedback):
    messages = [
        {"role": "system", "content": "You are an Psychologist conducting research and gathering relevant research."},
        {"role": "user", "content": f"Improve the following summary: {summary} based on the following feedback: {feedback}"}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500,
        stop=None,
        temperature=0.7,
    )
    summ = response.choices[0].message.content
    return summ

In [87]:
def research_helper_agent():
  research_topic = input("Welcome! I am a research agent that can help gather relevant topics to your research and provide you with valueable summaries.\n\nPlease enter your research topic:\n\n\nRESEARCH TOPIC:")
  print("\n#### 1. Breaking down research topic into subtopics:")
  subtopics = generate_subtopics(research_topic)
  print(f"\n Generated the following subtopics:\n{subtopics}")

  print("\n\n\n#### 2. Expanding the topics for broarder search results:")
  expanded_queries = expand_queries(subtopics)
  print(f"\n Generated the following expansions for each topic:\n{expanded_queries}")

  print("\n\n\n#### 3. Searching the web for relevant studies :")
  web_search_results = get_ai_snippets_for_query(expanded_queries)
  print(f"\n Web Search Result:\n{web_search_results['hits']}")

  print("\n\n\n#### 4. Generating summary based on web search:")
  summary1 = summary_generator(web_search_results['hits'])
  print(f"\n Web Search Result:\n{summary1}")

  print("\n\n\n#### 5. Evaluating summary for correcness:")
  summary_eval = evaluate_search(summary1)
  print(f"\n Web Search Result:\n{summary_eval}")

  print("\n\n\n#### 6. Regenerating Summary based on self-eval:")
  summary2 = summary_generator_improve(summary1, summary_eval)
  print(f"\n Web Search Result:\n{summary2}")


In [88]:
research_helper_agent()

Welcome! I am a research agent that can help gather relevant topics to your research and provide you with valueable summaries.

Please enter your research topic:


RESEARCH TOPIC:Impact of Human-Dog Interaction on Well-being

#### 1. Breaking down research topic into subtopics:

 Generated the following subtopics:
1. The benefits of having a dog for mental health
2. The effects of different breeds of dogs on human well-being
3. The impact of pet ownership on physical health
4. The role of dog communication in human-dog interactions and well-being
5. The impact of dog training on human well-being



#### 2. Expanding the topics for broarder search results:

 Generated the following expansions for each topic:
1. The advantages of having a dog for mental health
  2. The consequences of different types of dogs on human health
  3. The effect of pet ownership on physical fitness
  4. The importance of dog communication in human-dog interactions and wellness
  5. The impact of dog training o